# Load the necessary modules

In [ ]:
import os
import logging
from dotenv import load_dotenv

from agent_framework.azure import AzureOpenAIChatClient
from agent_framework import ChatAgent

from tools import SqlDatabase

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)

# Read the necessary environment variables

In [ ]:
logger.info("Hello from agent-framework-minimal-sql!")

load_dotenv()

# necessary to connect to the Azure OpenAI models
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")


# Necessary to connect to the Azure SQL Database (or the SQL dedicated pool in Azure Synapse)
connection_string = os.environ.get("AZURE_SQL_CONNECTIONSTRING")
if not connection_string or not endpoint or not deployment_name or not api_key:
    raise ValueError("AZURE_SQL_CONNECTIONSTRING environment variable is required")

In [ ]:
# Debug: Check if environment variables are loaded correctly
print(f"Endpoint: {endpoint}")
print(f"Deployment: {deployment_name}")
print(f"API Key exists: {bool(api_key)}")
print(f"API Version: {api_version}")

## Create the necessary classes and objects

In [ ]:
load_dotenv()

# necessary to connect to the Azure OpenAI models
endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
deployment_name = os.getenv("AZURE_OPENAI_MODEL")
api_key = os.getenv("AZURE_OPENAI_API_KEY")
api_version = os.getenv("AZURE_OPENAI_API_VERSION")


# this will enable the llm connection to Azure OpenAI
llm = AzureOpenAIChatClient(
    endpoint=endpoint or None,
    deployment_name=deployment_name or None,
    api_key=api_key or None,
    api_version=api_version or None,
)

# response = await llm.get_response("Hello, world!")  # test the connection
# response.text

## define the ai search integration

In [ ]:
search_endpoint = os.getenv("SEARCH_SERVICE_ENDPOINT")
index_name = os.getenv("SEARCH_INDEX_NAME")

In [ ]:
if "azure.us" in search_endpoint:
    cloud_name = "AzureUSGovernment"
else:
    cloud_name = "AzurePublicCloud"
print(f"Using cloud: {cloud_name}")

In [ ]:
from rag_tools import semantic_search, list_facets

results = await semantic_search("test")

print(results)

## For the ai search agent

In [ ]:
# this will enable the llm connection to Azure OpenAI
llm = AzureOpenAIChatClient(
    endpoint=endpoint or None,
    deployment_name=deployment_name or None,
    api_key=api_key or None,
    api_version=api_version or None,
)

# this creates the agent
# An agent can be thought of as a chat client with a set of tools it
# can use to support the user in its response
rag_agent = ChatAgent(
    chat_client=llm,
    name="agent_name",
    instructions="Your instructions here",  # instructions to guide the agent behavior, aka the system prompt
    tools=[
        semantic_search,
        list_facets,
    ],  # the tools the agent can use
    temperature=0.2,  # the lower the temperature, the more deterministic the responses
)

# this creates a new chat thread, it is responsible for managing the
# state of the conversation, including the history of messages, and tools used
# If run without a thread, the agent will not be able to remember previous
# messages or tool calls
thread = rag_agent.get_new_thread()


query = "what is a class 2 drone."

In [ ]:
async for msg in rag_agent.run_stream(query, thread=thread):
    print(f"Streamed message: {msg.to_dict()}")

In [ ]:
result = await rag_agent.run(query, thread=thread)
print(f"Agent response: {result}")

## for the SQL agent

In [ ]:
# this class creates the connection and exposes some methods as tools
db_tool = SqlDatabase(connection_string)

# this will enable the llm connection to Azure OpenAI
llm = AzureOpenAIChatClient(
    endpoint=endpoint or None,
    deployment_name=deployment_name or None,
    api_key=api_key or None,
    api_version=api_version or None,
)

# this creates the agent
# An agent can be thought of as a chat client with a set of tools it
# can use to support the user in its response
sql_agent = ChatAgent(
    chat_client=llm,
    name="agent_name",
    instructions="Your instructions here",  # instructions to guide the agent behavior, aka the system prompt
    tools=[
        db_tool.list_tables,
        db_tool.describe_table,
        db_tool.read_query,
    ],  # the tools the agent can use
    temperature=0.2,  # the lower the temperature, the more deterministic the responses
)

# this creates a new chat thread, it is responsible for managing the
# state of the conversation, including the history of messages, and tools used
# If run without a thread, the agent will not be able to remember previous
# messages or tool calls
thread = sql_agent.get_new_thread()

# Run the agent with a query

In [ ]:
query = "List the top 5 airlines by total aircraft."
result = await sql_agent.run(query, thread=thread)
print(f"Agent response: {result}")

In [ ]:
# async for msg in sql_agent.run_stream(query, thread=thread):
#     print(f"Streamed message: {msg.to_dict()}")